
<div class="markdown-google-sans">

## **Fashin Image Recommender Bot. Deep Learning Image Recommender System with ResNet50**
</div>

The goal of this project is to create a sophisticated image recommender system that leverages the power of deep learning. The system utilizes the ResNet50 architecture, a deep convolutional neural network, to extract meaningful features from images and provide accurate recommendations.

<div class="markdown-google-sans">

## **Install Dependencies**
</div>



In [ ]:
pip install -r requirements.txt

<div class="markdown-google-sans">

## **Download Dataset**
</div>


In [ ]:
git clone https://github.com/alexeygrigorev/clothing-dataset.git

or

In [ ]:
 wget https://github.com/alexeygrigorev/clothing-dataset/archive/refs/heads/master.zip
 unzip master.zip

move this images folder into current project directory

<div class="markdown-google-sans">

## **Training the Model**
</div>



1.  **Importing Libraries:**

In [ ]:
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import numpy as np
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle



*   **TensorFlow** is imported as tensorflow.
*   Specific modules from TensorFlow and Keras are imported for image preprocessing and ResNet50 model.
*   **numpy** is imported as np for numerical operations.
*   **os** is imported for operating system-related functions.
*   **tqdm** is used to show progress bars during iterations.
*   **pickle** is used for serializing and deserializing Python objects.



2.   **Loading Pre-trained ResNet50 Model:**

In [ ]:
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model.trainable = False



*   The ResNet50 model is loaded with pre-trained ImageNet weights.
*   The `include_top=False` parameter excludes the fully connected layers at the top of the network.

*   The input shape is set to (224, 224, 3).
*   The model is set to be non-trainable (trainable=False) to use it as a fixed feature extractor.



3. **Creating a Sequential Model:**






In [ ]:
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])



*   The ResNet50 model is wrapped in a new sequential model, followed by a GlobalMaxPooling2D layer.
*   Global max pooling reduces the spatial dimensions of the input tensor while retaining important information.



4.  **Function to Extract Features from an Image:**


In [ ]:
def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    return normalized_result



*   This function takes an image path and the ResNet50 model as inputs.
*   It loads and preprocesses the image, then extracts features from the model's output.
*   The resulting feature vector is normalized using L2 normalization.


In [ ]:
filenames = []

for file in os.listdir('images'):
    filenames.append(os.path.join('images',file))

feature_list = []

for file in tqdm(filenames):
    feature_list.append(extract_features(file,model))

pickle.dump(feature_list,open('embeddings.pkl','wb'))
pickle.dump(filenames,open('filenames.pkl','wb'))



*   This creates a list of file paths for all the images in the 'images' directory.
*   Features are extracted for each image using the extract_features function.

*   The extracted features and corresponding filenames are saved using the pickle module. The features are saved in 'embeddings.pkl', and filenames are saved in 'filenames.pkl'. Both files are written in binary mode ('wb').

run bellow command to generate  `embeddings.pkl` and `filenames.pkl` files

In [ ]:
python3 app.py

<div class="markdown-google-sans">

## **Streamlit application for an Image Recommender System**
</div>



1.  **Importing Libraries:**

In [ ]:
import streamlit as st
import os
from PIL import Image
import numpy as np
import pickle
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.neighbors import NearestNeighbors
from numpy.linalg import norm



2.   **Loading Pre-trained ResNet50 Model and Feature Vectors:**

In [ ]:
feature_list = np.array(pickle.load(open('embeddings.pkl', 'rb')))
filenames = pickle.load(open('filenames.pkl', 'rb'))

model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model.trainable = False

model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])



3.   **File Upload and Image Display:**


*   Uses Streamlit's file_uploader to allow the user to upload an image.
*   If an image is uploaded successfully, it is displayed on the web page using st.image.




In [ ]:
uploaded_file = st.file_uploader("Choose an image")
if uploaded_file is not None:
    if save_uploaded_file(uploaded_file):
        display_image = Image.open(uploaded_file)
        st.image(display_image)


4.   **Feature Extraction and Recommendation:**

In [ ]:
features = feature_extraction(os.path.join("uploads", uploaded_file.name), model)
indices = recommend(features, feature_list)



5.   **Utility Functions:**



In [ ]:
def save_uploaded_file(uploaded_file):
    try:
        with open(os.path.join('uploads',uploaded_file.name),'wb') as f:
            f.write(uploaded_file.getbuffer())
        return 1
    except:
        return 0

def feature_extraction(img_path,model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    return normalized_result

def recommend(features,feature_list):
    neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
    neighbors.fit(feature_list)

    distances, indices = neighbors.kneighbors([features])




6.   Displaying Recommended Images:




In [ ]:
col1, col2, col3, col4, col5 = st.columns(5)

with col1:
    st.image(filenames[indices[0][0]])
with col2:
    st.image(filenames[indices[0][1]])
with col3:
    st.image(filenames[indices[0][2]])
with col4:
    st.image(filenames[indices[0][3]])
with col5:
    st.image(filenames[indices[0][4]])



*   The recommend function is used to find the indices of the nearest neighbors based on the uploaded image's features.
*   The top 5 recommended images are displayed in five columns using Streamlit's st.image.



7.  **Run bellow command to start Streamlit application**

```
streamlit run main.py
```


9.  You can now view your Streamlit app in your browser with port 8501

    Link : http://localhost:8501/